<a href="https://colab.research.google.com/github/akib1162100/ML_base/blob/main/Pretrained_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch transformers pandas scikit-learn
!pip install datasets

In [ ]:
!pip install Dataset

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
from datasets import load_dataset
import torch
import random

In [ ]:
dataset = load_dataset("wikitext", "wikitext-2-raw-v1")
texts = dataset["train"]["text"]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
#shuffle and slice dataset
shuffled_indices = list(range(len(texts)))
random.shuffle(shuffled_indices)
selected_indices = shuffled_indices[:200]

In [ ]:
train_texts = [texts[i] for i in selected_indices]
train_text = " ".join(train_texts)

In [ ]:
from torch.utils.data import Dataset
class LanguageModelingDataset(Dataset):
    def __init__(self, text, tokenizer, max_length):
        self.input_ids = tokenizer.encode(text, return_tensors="pt", max_length=max_length)

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {"input_ids": self.input_ids[idx], "labels": self.input_ids[idx].clone()}


In [ ]:
model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

In [ ]:
input_ids = tokenizer(train_text, return_tensors="pt")["input_ids"]
labels = input_ids[:, 1:].contiguous()

Token indices sequence length is longer than the specified maximum sequence length for this model (13997 > 1024). Running this sequence through the model will result in indexing errors


In [ ]:
! pip install -U accelerate
! pip install -U transformers

In [ ]:
max_length = 512  # Adjust as needed
custom_dataset = LanguageModelingDataset(train_text, tokenizer, max_length)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [ ]:
training_args = TrainingArguments(
    per_device_train_batch_size=2,
    output_dir="./gpt2_next_word_prediction",
    num_train_epochs=3,
    logging_steps=100,
    save_steps=500,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=custom_dataset,
)
trainer.train()

Step,Training Loss


TrainOutput(global_step=3, training_loss=3.909398396809896, metrics={'train_runtime': 51.7861, 'train_samples_per_second': 0.058, 'train_steps_per_second': 0.058, 'total_flos': 783876096000.0, 'train_loss': 3.909398396809896, 'epoch': 3.0})

In [ ]:
model.save_pretrained("./gpt2_next_word_prediction")
tokenizer.save_pretrained("./gpt2_next_word_prediction")

('./gpt2_next_word_prediction/tokenizer_config.json',
 './gpt2_next_word_prediction/special_tokens_map.json',
 './gpt2_next_word_prediction/vocab.json',
 './gpt2_next_word_prediction/merges.txt',
 './gpt2_next_word_prediction/added_tokens.json')

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load the fine-tuned GPT-2 model and tokenizer
fine_tuned_model_path = "./gpt2_next_word_prediction"
model = GPT2LMHeadModel.from_pretrained(fine_tuned_model_path)
tokenizer = GPT2Tokenizer.from_pretrained(fine_tuned_model_path)

# Set the model to evaluation mode
model.eval()

# Generate text with the model
prompt = "The quick brown fox"
input_ids = tokenizer.encode(prompt, return_tensors="pt")

# Generate text using the model
output_ids = model.generate(input_ids, max_length=50, num_return_sequences=1, temperature=0.8)

# Decode and print the generated text
generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print("Generated Text:", generated_text)


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Text: The quick brown foxes are a common sight in the wild, but they are also found in the wild in the United States. They are also found in the wild in the United States. They are also found in the wild in the United States.
